# GMSO - General Molecular Simulation Object

## Modifying a topology

[GMSO](https://github.com/mosdef-hub/gmso) allows for engine agnostic python object to store everything needed for writing molecule input files for simulation.


This notebook is designed to provide advanced principles for interfacing with GMSO. Tutorials get more in-depth in terms of complex usage denoted by the `beginners`, `intermediates`, and `experts` tags on these notebooks. For a full list of the examples discussed in these tutorials, please see [the GMSO Tutorials README](README.md).
This is the last notebook directly related to `GMSO`. See the forcefields tutorials to learn more about writing GMSO forcefields.

Contained in this notebook is examples of:
* Working with unyts
* Converting parameters
* Modifying a topology charge
* Identifying molecules in the topology

In [ ]:
# imports
import copy
import time
import numpy as np
import warnings
warnings.simplefilter('ignore')

import gmso
from gmso.parameterization import apply

## Working with `Unyts`
For details go the [documentation](https://unyt.readthedocs.io/en/stable/) page. Unyts are a set of python objects for keeping track of the units a value is associated with, and giving the opportunity to convert them to equivalent values.

In [ ]:
top = gmso.Topology.load("source_files/ethane-box.json")
m = top.sites[0].mass
print(m)

In [ ]:
m.in_units("g")

In [ ]:
# calculate density of the simulation box
mass = 0
for site in top.sites:
    mass += site.mass #can do summation, multiplication, etc just like floats
volume = 1
for dim in top.box.lengths:
    volume *= dim 
density = mass/volume
print(density.in_units("g/cm**3"))
print(density.value) # decouple float
print(density.units) # get the units

In [ ]:
# gmso units
from gmso.utils.units import LAMMPS_UnitSystems
base_unyts = LAMMPS_UnitSystems("real")
base_unyts.usystem

In [ ]:
import unyt as u

init_value = 1 * u.kJ/u.mol 
init_value.in_units(base_unyts.usystem["energy"], reg=base_unyts.reg)

In [ ]:
import unyt as u

init_value = 1 * u.kJ/u.mol 
base_unyts.convert_parameter(init_value)

## Converting parameters
In regards to generating forcefields, large scale conversions of parameters from literature by hand can be error prone. Because of this, sometimes it's necessary to use parameters from the forcefield given in one formula, and convert to something else more useful. We will do this using the dihedral rb-torsion to opls conversion.

In [ ]:
top = gmso.Topology.load("source_files/ethane-typed.json")
list(top.dihedral_types)[0]

In [ ]:
top_conv = top.convert_potential_styles({"dihedrals":"OPLSTorsionPotential"})
list(top.dihedral_types)[0]

To see other possible conversions, go to the conversion utility of [GMSO](https://github.com/mosdef-hub/gmso/blob/bde9f1b8fcdcb7fd62273215922b932b1d75648f/gmso/utils/conversions.py#L53) to look at built in conversions, automated algebraic conversions such as one half factors, and how to add new conversions"

## Modifying a topology charge

In [ ]:
top = gmso.Topology.load("source_files/ethane-typed.json")
charge0 = 0
for site in top.sites:
    if site.atom_type.charge < 0:
        charge0+=site.atom_type.charge
    else:
        charge0-=site.atom_type.charge
print(charge0)
charge_new = 0
new_scalar = 0.8
for site in top.sites:
    site.atom_type.charge = site.atom_type.charge * new_scalar #
    if site.atom_type.charge < 0:
        charge_new+=site.atom_type.charge
    else:
        charge_new-=site.atom_type.charge
print(charge_new)     
print(charge_new/charge0)

## Identifying molecules in the topology

In [ ]:
top = gmso.Topology.load("source_files/mixed-box.json")
molecules = top.unique_site_labels(name_only=True)
print(molecules)
moleculeDict = {}
for molecule in top.unique_site_labels(name_only=False):
    if molecule.name in moleculeDict:
        moleculeDict[molecule.name]+=1
    else:
        moleculeDict[molecule.name] = 1
moleculeDict

Code used to generate this topology
```python
import mbuild as mb
cpd1 = mb.load("O", smiles=True)
cpd1.name = "water"
cpd2 = mb.load("CCCCCCCCO", smiles=True)
cpd2.name = "octanol"
cpd3 = mb.load("C(C1C(C(C(C(O1)O)O)O)O)O", smiles=True)
cpd3.name = "glucose"

box = mb.fill_box([cpd1, cpd2, cpd3], n_compounds=[1000, 500, 50], density=0.831)
top = box.to_gmso()
top.save("source_files/mixed-box.json")
```